In [6]:
from binance.client import Client
from binance.enums import *
from time import time
import pickle as pickle
from datetime import datetime
import pandas as pd

def get_client():
    fn = '../../key/binance-key.pickle'
    # fn = '/home/era/key/binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


client = get_client()


def get_unix_timestamp(date_string):
    """
    Converts the input date string to Unix timestamp.

    Parameters:
        date_string (str): Input date string in the format "dd/mm/yyyy hh:mm:ss".

    Returns:
        int: Unix timestamp of the given date.
    """
    try:
        date_obj = datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S")
        timestamp = int(date_obj.timestamp())
        return timestamp
    except ValueError:
        print("Invalid date format. Please use the format 'dd/mm/yyyy hh:mm:ss'.")
        return None

def get_historical_data(start_timestamp, end_timestamp): 
    data = []
    tot = (end_timestamp - start_timestamp)/(900*500)
    cntr = 0
    for current_sts in range(start_timestamp, end_timestamp+1, 900*500):
        next_ets = current_sts + 900*500 if (current_sts + 900*500) < end_timestamp else end_timestamp
        print(current_sts, next_ets, f'100% completed') if next_ets == end_timestamp else print(current_sts, next_ets, f'{round(cntr*100/tot, 1)}% completed')
        cntr += 1
        # Futures market
        klines = client.futures_historical_klines(coin_pair, interval, current_sts*1000, next_ets*1000, limit=500)
        # Spot market
        # klines = client.get_historical_klines(coin_pair, interval, current_sts*1000, next_ets*1000, limit=500)
        
        for kline in klines:
            timestamp = kline[0]/1000
            open_price = float(kline[1])
            high_price = float(kline[2])
            low_price = float(kline[3])
            close_price = float(kline[4])
            volume = float(kline[5])

            data.append([timestamp, open_price, high_price, low_price, close_price, volume])

    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    df.to_csv(f'../../data/{coin_pair}-{interval}-validate-fetares.csv', index=False)
    print('Data Exported')

coin_pair = 'ETHUSDT'
interval = '15m'

# Thursday, August 17, 2017 4:00:00 AM
# start_timestamp = get_unix_timestamp('1/1/2016 00:00:00')
# end_timestamp = int(time())
# end_timestamp = get_unix_timestamp('1/1/2022 00:00:00')
start_timestamp = 1692215100-900
end_timestamp = 1692665100

get_historical_data(start_timestamp, end_timestamp)


1692214200 1692664200 0.0% completed
1692664200 1692665100 100% completed
Data Exported


In [8]:
import pandas as pd
import pandas_ta as ta
import random



candlestick_frame = 12
pnl_threshold = 3


df = pd.read_csv('../../data/ETHUSDT-15m-validate-fetares.csv')

try:
    df.ta.strategy("all")
except Exception as e:
    print(e)

    ts = list(df['time'])
    open = list(df['open'])
    high = list(df['high'])
    low = list(df['low'])
    close = list(df['close'])
    volume = list(df['volume'])
    tot = len(ts)
    long_runup_lst = []
    long_drawdown_lst = []
    short_runup_lst = []
    short_drawdown_lst = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            max_high = max(high[idx+1:idx+candlestick_frame])
            min_low = min(low[idx+1:idx+candlestick_frame])
            entry_price = open[idx+1]
            long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
            long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
            short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
            short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
        else:
            long_runup_lst.append(0)
            long_drawdown_lst.append(0)
            short_runup_lst.append(0)
            short_drawdown_lst.append(0)     


    long=[]
    short=[]
    dont_trade=[]
    signal = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            if long_runup_lst[idx] >= pnl_threshold:
                signal.append('long')
            elif short_runup_lst[idx] >= pnl_threshold:
                signal.append('short')
            else:
                signal.append('dont_trade')
        else:
            signal.append('dont_trade')


    df['signal'] = signal

    # long_indices = df[df['signal'].str.contains('long', case=False)].index
    # short_indices = df[df['signal'].str.contains('short', case=False)].index
    # dont_trade_indices = list(df[df['signal'].str.contains('dont_trade', case=False)].index)
    # num_indices_to_pick  = len(dont_trade_indices) - min([len(long_indices), len(short_indices)])
    # random_indices = random.sample(dont_trade_indices, num_indices_to_pick)
    # df = df.drop(random_indices)

    # df.reset_index(drop=True, inplace=True)

    # df = df.drop(columns = ['time', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    
    print('saving dataset.csv....')
    dataset_fn = f'../../data/dataset-ETHUSDT-15m-validate-fetares.csv'
    pd.DataFrame(df).to_csv(dataset_fn, index=False)




0it [00:00, ?it/s]

[!] VWAP volume series is not datetime ordered. Results may not be as expected.
[!] VWAP price series is not datetime ordered. Results may not be as expected.


/usr/lib/python3.10/multiprocessing/pool.py:48: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return list(map(*args))
123it [00:00, 611.49it/s]

'RangeIndex' object has no attribute 'to_period'
saving dataset.csv....
